In [1]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT
import pandas as pd
from shapely.wkt import loads
from shapely.geometry import mapping
import tracers as tr
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from shapely.geometry import LineString
import json
import os

#load network data and graph 
network = Network("../osmnx_example/rome/edges.shp","fid", "u", "v")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

#load UBODT
ubodt = UBODT.read_ubodt_csv("../osmnx_example/rome/ubodt.txt")

Nodes 88690 edges 188608[2025-03-24 18:28:57.393] [info] [network.cpp:72] Read network from file ../osmnx_example/rome/edges.shp

[2025-03-24 18:28:59.351] [info] [network.cpp:172] Number of edges 188608 nodes 88690
[2025-03-24 18:28:59.351] [info] [network.cpp:173] Field index: id 20 source 0 target 1
[2025-03-24 18:28:59.532] [info] [network.cpp:176] Read network done.
[2025-03-24 18:28:59.533] [info] [network_graph.cpp:17] Construct graph from network edges start
[2025-03-24 18:28:59.561] [info] [network_graph.cpp:30] Graph nodes 88690 edges 188608
[2025-03-24 18:28:59.561] [info] [network_graph.cpp:31] Construct graph from network edges end
[2025-03-24 18:28:59.563] [info] [ubodt.cpp:208] Reading UBODT file (CSV format) from ../osmnx_example/rome/ubodt.txt
[2025-03-24 18:29:02.171] [info] [ubodt.cpp:236] Read rows 1000000
[2025-03-24 18:29:03.225] [info] [ubodt.cpp:236] Read rows 2000000
[2025-03-24 18:29:04.222] [info] [ubodt.cpp:236] Read rows 3000000
[2025-03-24 18:29:05.172] [i

In [4]:
METER_PER_DEGREE = 109662.80313373724

with open('datapath.json', 'r') as f:
    data = json.load(f)

root_path = data['data_path']
user_path = 'taxi_21/taxi-21-rit-2.csv'
file = os.path.join(root_path, user_path)

df = pd.read_csv(file, sep=",")
df = df.rename(columns={"ID": "id", "Latitude": "x", "Longitude": "y", "Timestamp": "timestamp"})

# Convert timestamp to epoch and sort by timestamp
df["timestamp"] = pd.to_datetime(df["timestamp"], format="ISO8601", errors="coerce").astype(int) / 10**9
df["timestamp"] = df["timestamp"].astype(int)
df = df.sort_values(by="timestamp")
trace = df[['y', 'x', 'timestamp']].to_numpy()
trace = [tuple(x) for x in trace]
trace = [(x[0], x[1], int(x[2])) for x in trace]
perturbed_traces = tr.perturb_traces((50, 30), [trace], picker_str='closest')
trace = [(x[0], x[1]) for x in trace]
perturbed_trace = perturbed_traces[0]    

# Create wkt from lat lon data
original_linestring = LineString(zip(df["y"], df["x"]))
original_wkt = original_linestring.wkt
model = FastMapMatch(network, graph, ubodt)

# Configuration params
k = 7
radius = 300 / METER_PER_DEGREE
gps_error = 15 / METER_PER_DEGREE

fmm_config = FastMapMatchConfig(k, radius, gps_error, perturbation=False, reverse_tolerance=0.1)

# mm on the original trajectory 
result = model.match_wkt(original_wkt, fmm_config)


#mm on perturbed trajectory
df_pert = pd.DataFrame(perturbed_trace, columns=["y", "x", "timestamp"])
df_pert["id"] = df["id"].iloc[0]
perturbed_linestring = LineString(zip(df_pert["y"], df_pert["x"]))
perturbed_wkt = perturbed_linestring.wkt

pert_stdev = 50/METER_PER_DEGREE
k = 20

fmm_config_pert = FastMapMatchConfig(k, radius, pert_stdev, perturbation=True, reverse_tolerance=0.1)
result_pert = model.match_wkt(perturbed_wkt, fmm_config_pert)

gt_geom = loads(result.mgeom.export_wkt())
mm_geom = loads(result_pert.mgeom.export_wkt())
intersection = gt_geom.intersection(mm_geom).length
gt_length = gt_geom.length
mm_length = mm_geom.length
if mm_length > 0 and gt_length > 0: 
     accuracy = intersection / max(mm_length, gt_length)
     print(f"accuracy: {accuracy}")

accuracy: 0.7133973194146248


In [5]:
import folium
from shapely.wkt import loads
from folium.plugins import MarkerCluster

# Define slice range
start_idx = 0
end_idx = len(gt_geom.coords)

# Load original trace
original_geom = loads(original_wkt)

# Initialize map at the first sliced point
m = folium.Map(location=[original_geom.coords[start_idx][1], original_geom.coords[start_idx][0]], zoom_start=16)

# Plot sliced original trace
folium.PolyLine([(lat, lon) for lon, lat in original_geom.coords[start_idx:end_idx]], 
                color="red", weight=3).add_to(m)

for i, (lon, lat) in enumerate(original_geom.coords[start_idx:end_idx], start=start_idx):
    folium.CircleMarker(location=[lat, lon], radius=3, color="red", fill=True, fill_color="red",
                        popup=f"Original Point {i}").add_to(m)

# Matched trace
if result.cpath:
    matched_geom = loads(result.mgeom.export_wkt())

    folium.PolyLine([(lat, lon) for lon, lat in matched_geom.coords], 
                    color="blue", weight=3, opacity=0.8, tooltip="Matched Path").add_to(m)

# Perturbed trace
pert_geom = loads(perturbed_wkt)
folium.PolyLine([(lat, lon) for lon, lat in pert_geom.coords[start_idx:end_idx]], 
                color="green", weight=3).add_to(m)

for i, (lon, lat) in enumerate(pert_geom.coords[start_idx:end_idx], start=start_idx):
    folium.CircleMarker(location=[lat, lon], radius=3, color="green", fill=True, fill_color="green",
                        popup=f"Perturbed Point {i}").add_to(m)

# Matched perturbed trace
if result_pert.cpath:
    matched_geom = loads(result_pert.mgeom.export_wkt())
    matched_points = loads(result_pert.pgeom.export_wkt())

    folium.PolyLine([(lat, lon) for lon, lat in matched_geom.coords], 
                    color="yellow", weight=3, opacity=0.8, tooltip="Matched Perturbed Path").add_to(m)

    # Use a marker cluster with better settings
    marker_cluster = MarkerCluster(options={"disableClusteringAtZoom": 30}).add_to(m)

    # Plot only sliced range for matched perturbed points
    for i in range(start_idx, min(end_idx, len(matched_points.coords))):  
        lon, lat = matched_points.coords[i]

        popup_text = f"MM pt. {i} <br> tp: {result_pert.candidates[i].tp} <br> ep: {result_pert.candidates[i].ep} <br> sp: {result_pert.candidates[i].spdist * METER_PER_DEGREE}"

        folium.CircleMarker(
            location=[lat, lon], 
            radius=3, 
            color="purple", 
            fill=True, 
            fill_color="purple", 
            fill_opacity=0.8,
            popup=popup_text
        ).add_to(marker_cluster)

print(len(matched_geom.coords))

# Display map
m


1950
